<a href="https://colab.research.google.com/github/alfianhid/Prediksi-Churn-Rate-Pada-Sebuah-Bank-Menggunakan-PySpark/blob/main/Prediksi_Churn_Rate_Pada_Sebuah_Bank_Menggunakan_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pertama, kita perlu install JDK karena program/aplikasi Spark berjalan di JVM (berbasis Java)**

In [ ]:
# Saya menggunakan JDK 8 karena alasan lebih stabil
!apt-get install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 15 newly installed, 0 to remove and 11 not upgraded.
Need to get 43.5 MB of archives.
After this 

**Setelah install JDK, kita perlu menge-set path environment Java supaya kodingan kita di Spark bisa berjalan**

In [ ]:
import os # library OS berfungsi menjembatansi proses/tugas antara kodingan dengan sistem operasi
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64" # path folder JDK bisa kita dapatkan dari output saat install JDK
!echo $JAVA_HOME # cek kembali apakah sudah di-set dengan benar

/usr/lib/jvm/java-8-openjdk-amd64


**Setelah JDK terinstall dengan benar, baru kita install Spark-nya. Di sini, saya menggunakan PySpark (ngoding Spark pake Python)**

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 69kB/s 
     |████████████████████████████████| 204kB 16.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=9284d8eb67b4e9bb4ceb561ede5646d9549157ac520bc19e6c52a0a69c258f7c
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


**Selanjutnya kita perlu mengintegrasikan Google Colab dengan Google Drive karena di sana tempat kita menyimpan dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # path folder default Google Drive di Google Colab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Lalu, kita wajib men-start session di Spark sebelum ngoding. Ini berfungsi agar kita bisa menjalankan library-library Spark dengan lancar.**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Analisis Data Nasabah Bank').getOrCreate()
# Sumber Dataset: https://archive.ics.uci.edu/ml/datasets/bank+marketing (UCI Machine Learning Repository)

# **Alur Kerja**
# Real Data >> Data Preparation >> Data Cleaning >> Data Transformation >> Data Modeling >> Prediction Process with Machine Learning

**Kemudian tinggal kita lakukan "read data" terhadap dataset yang telah kita upload di Google Drive**

In [ ]:
# "inferSchema=True" berfungsi agar kita bisa menampilkan informasi dari dataframe kita nanti
# "header=True" berfungsi agar baris pertama pada dataset diubah menjadi header dalam dataframe
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Datasets/dataset-nasabah-bank.csv',inferSchema=True,header=True)

**Setelah itu, kita bisa mencoba melihat dimensi data atau seberapa banyak data pada dataset yang kita miliki**

In [ ]:
print((df.count(),len(df.columns))) # baris x kolom

(11162, 17)


**Dilanjutkan dengan mem-print Schema dari dataframe untuk mengetahui informasi dataframe kita**

In [ ]:
df.printSchema() # nullable=true artinya nilai/value pada variabel bisa kosong atau tanpa nilai

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



**Next, seperti biasa, kita tampilkan lima baris data teratas pada dataframe kita**

In [ ]:
df.show(5)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|  services|married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|    admin.|married| tertiary|     no|    184|     no|  no|unknown|  5| 

**Dari lima baris data tersebut, perlu dilakukan "data cleaning" pada kolom yang tak memiliki arti/makna supaya analisis data kita lebih efisien**

In [ ]:
my_data = df.drop(*['default', 'contact', 'day','month']) # menghapus kolom default, contact, day, dan month
my_data.columns # menampilkan sisa kolom setelah proses drop kolom dijalankan

['age',
 'job',
 'marital',
 'education',
 'balance',
 'housing',
 'loan',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit']

**Dan selanjutnya, kita bisa mencoba menampilkan informasi lain dari dataframe yang kita miliki untuk lebih mengenal data kita**

In [ ]:
my_data.describe().show()

+-------+------------------+-------+--------+---------+------------------+-------+-----+------------------+------------------+------------------+------------------+--------+-------+
|summary|               age|    job| marital|education|           balance|housing| loan|          duration|          campaign|             pdays|          previous|poutcome|deposit|
+-------+------------------+-------+--------+---------+------------------+-------+-----+------------------+------------------+------------------+------------------+--------+-------+
|  count|             11162|  11162|   11162|    11162|             11162|  11162|11162|             11162|             11162|             11162|             11162|   11162|  11162|
|   mean|41.231947679627304|   null|    null|     null|1528.5385235620856|   null| null|371.99381831213043| 2.508421429851281| 51.33040673714388|0.8325568894463358|    null|   null|
| stddev|11.913369192215518|   null|    null|     null| 3225.413325946149|   null| null|34

**Nah, dari summary di atas, kita bisa melihat bahwa masih banyak kolom yang memiliki nilai berupa null. Maka dari itu, kita perlu melakukan normalisasi data dengan menghapus data yang memiliki nilai null tersebut.**

In [ ]:
df.na.drop(subset=["job","marital","education","housing","loan","poutcome","deposit"]) \ # menghapus baris data null hanya pada kolom terpilih
   .show(truncate=False)

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job        |marital |education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|59 |admin.     |married |secondary|no     |2343   |yes    |no  |unknown|5  |may  |1042    |1       |-1   |0       |unknown |yes    |
|56 |admin.     |married |secondary|no     |45     |no     |no  |unknown|5  |may  |1467    |1       |-1   |0       |unknown |yes    |
|41 |technician |married |secondary|no     |1270   |yes    |no  |unknown|5  |may  |1389    |1       |-1   |0       |unknown |yes    |
|55 |services   |married |secondary|no     |2476   |yes    |no  |unknown|5  |may  |579     |1       |-1   |0       |unknown |yes    |
|54 |admin.     |married |tertiary |no     |184    |no     |no

**Setelah berhasil menormalisasi data, kita lanjutkan lagi eksplorasi dataframe kita supaya hubungan terasa lebih mesra**

In [ ]:
# Di sini, kita akan menghitung jumlah data (baris) pada setiap kategori dari sebuah variabel data
my_data.groupBy('job').count().show()
print()
my_data.groupBy('marital').count().show()
print()
my_data.groupBy('education').count().show()
print()
my_data.groupBy('loan').count().show()
print()
my_data.groupBy('poutcome').count().show()
print()
my_data.groupBy('deposit').count().show()

+-------------+-----+
|          job|count|
+-------------+-----+
|   management| 2566|
|      retired|  778|
|      unknown|   70|
|self-employed|  405|
|      student|  360|
|  blue-collar| 1944|
| entrepreneur|  328|
|       admin.| 1334|
|   technician| 1823|
|     services|  923|
|    housemaid|  274|
|   unemployed|  357|
+-------------+-----+


+--------+-----+
| marital|count|
+--------+-----+
|divorced| 1293|
| married| 6351|
|  single| 3518|
+--------+-----+


+---------+-----+
|education|count|
+---------+-----+
|  unknown|  497|
| tertiary| 3689|
|secondary| 5476|
|  primary| 1500|
+---------+-----+


+----+-----+
|loan|count|
+----+-----+
|  no| 9702|
| yes| 1460|
+----+-----+


+--------+-----+
|poutcome|count|
+--------+-----+
| success| 1071|
| unknown| 8326|
|   other|  537|
| failure| 1228|
+--------+-----+


+-------+-----+
|deposit|count|
+-------+-----+
|     no| 5873|
|    yes| 5289|
+-------+-----+



**Akhirnya kita sampai di pintu gerbang akhir Data Pre-processing. Mulai dari sini, kita akan memfungsikan dataset kita untuk sebuah fungsi prediksi. Adapun yang akan kita prediksi adalah "churn rate" dari setiap nasabah bank. Churn rate adalah persentase nasabah yang berhenti menggunakan produk dan layanan bank. Ini sangat penting untuk dianalisis karena menjadi acuan dalam menentukan masa depan bank.**

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Membuat objek dari StringIndexer class dan menge-set kolom input & output
SI_job = StringIndexer(inputCol='job',outputCol='job_Index')
SI_marital = StringIndexer(inputCol='marital',outputCol='marital_Index')
SI_education = StringIndexer(inputCol='education',outputCol='education_Index')
SI_housing = StringIndexer(inputCol='housing',outputCol='housing_Index')
SI_loan = StringIndexer(inputCol='loan',outputCol='loan_Index')
SI_poutcome = StringIndexer(inputCol='poutcome',outputCol='poutcome_Index')
SI_deposit = StringIndexer(inputCol='deposit',outputCol='deposit_Index')


# Mentransformasi / mengubah data ke dalam bentuk yang baru untuk mempermudah proses prediksi
my_data = SI_job.fit(my_data).transform(my_data)
my_data = SI_marital.fit(my_data).transform(my_data)
my_data = SI_education.fit(my_data).transform(my_data)
my_data = SI_housing.fit(my_data).transform(my_data)
my_data = SI_loan.fit(my_data).transform(my_data)
my_data = SI_poutcome.fit(my_data).transform(my_data)
my_data = SI_deposit.fit(my_data).transform(my_data)

**Lalu, kita lihat hasil transformasi data tadi. Di sini, kita akan melihat 10 baris data teratas dari dataframe.**

In [ ]:
my_data.select('job', 'job_Index', 'marital', 'marital_Index','housing','housing_Index','poutcome','poutcome_Index','deposit','deposit_Index').show(10)

+----------+---------+--------+-------------+-------+-------------+--------+--------------+-------+-------------+
|       job|job_Index| marital|marital_Index|housing|housing_Index|poutcome|poutcome_Index|deposit|deposit_Index|
+----------+---------+--------+-------------+-------+-------------+--------+--------------+-------+-------------+
|    admin.|      3.0| married|          0.0|    yes|          1.0| unknown|           0.0|    yes|          1.0|
|    admin.|      3.0| married|          0.0|     no|          0.0| unknown|           0.0|    yes|          1.0|
|technician|      2.0| married|          0.0|    yes|          1.0| unknown|           0.0|    yes|          1.0|
|  services|      4.0| married|          0.0|    yes|          1.0| unknown|           0.0|    yes|          1.0|
|    admin.|      3.0| married|          0.0|     no|          0.0| unknown|           0.0|    yes|          1.0|
|management|      0.0|  single|          1.0|    yes|          1.0| unknown|           0

**Setelah itu, kita akan mentransformasi data lagi ke dalam kolom OneHotEncoder sebelum seleksi fitur dari vektor biner**

In [ ]:
# Buat objek dan set kolom input & output
OHE = OneHotEncoder(inputCols=['job_Index', 'marital_Index','education_Index','housing_Index','loan_Index','poutcome_Index','deposit_Index'],outputCols=['job_OHE', 'marital_OHE','education_OHE','housing_OHE','loan_OHE','poutcome_OHE','deposit_OHE'])

# Proses transformasi data
my_data = OHE.fit(my_data).transform(my_data)

# Lihat hasil transformasi data
my_data.select('job', 'job_Index', 'job_OHE','education','education_Index','education_OHE').show(10)

+----------+---------+--------------+---------+---------------+-------------+
|       job|job_Index|       job_OHE|education|education_Index|education_OHE|
+----------+---------+--------------+---------+---------------+-------------+
|    admin.|      3.0|(11,[3],[1.0])|secondary|            0.0|(3,[0],[1.0])|
|    admin.|      3.0|(11,[3],[1.0])|secondary|            0.0|(3,[0],[1.0])|
|technician|      2.0|(11,[2],[1.0])|secondary|            0.0|(3,[0],[1.0])|
|  services|      4.0|(11,[4],[1.0])|secondary|            0.0|(3,[0],[1.0])|
|    admin.|      3.0|(11,[3],[1.0])| tertiary|            1.0|(3,[1],[1.0])|
|management|      0.0|(11,[0],[1.0])| tertiary|            1.0|(3,[1],[1.0])|
|management|      0.0|(11,[0],[1.0])| tertiary|            1.0|(3,[1],[1.0])|
|   retired|      5.0|(11,[5],[1.0])|secondary|            0.0|(3,[0],[1.0])|
|technician|      2.0|(11,[2],[1.0])|secondary|            0.0|(3,[0],[1.0])|
|  services|      4.0|(11,[4],[1.0])|secondary|            0.0|(

**Lalu, kita transformasi lagi menggunakan VectorAssembler supaya kita bisa melakukan seleksi fitur**

In [ ]:
from pyspark.ml.feature import VectorAssembler # men-transformasi list menjadi vektor biner

# menge-set kolom input & output dari transformasi data
assembler = VectorAssembler(inputCols=['age',
                                       'job_Index',
                                       'marital_Index',
                                       'education_Index',
                                       'balance',
                                       'housing_Index',
                                       'loan_Index',
                                       'duration',
                                       'campaign',
                                       'pdays',
                                       'previous',
                                       'poutcome_Index',
                                       'job_OHE',
                                       'marital_OHE',
                                       'housing_OHE',
                                       'education_OHE',
                                       'loan_OHE',
                                       'poutcome_OHE'],
                           outputCol='features')

# Proses transformasi data
final_data = assembler.transform(my_data)

**Lanjut seperti biasa, menampilkan 10 baris data teratas dari dataframe**

In [ ]:
final_data.select('features','deposit_Index').show(10)

+--------------------+-------------+
|            features|deposit_Index|
+--------------------+-------------+
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,4,7,8,9,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,3,4,7,8,...|          1.0|
|(33,[0,2,3,5,6,7,...|          1.0|
|(33,[0,3,4,5,6,7,...|          1.0|
|(33,[0,1,2,4,5,7,...|          1.0|
|(33,[0,1,4,5,7,8,...|          1.0|
|(33,[0,1,2,4,5,7,...|          1.0|
+--------------------+-------------+
only showing top 10 rows



**Nah, dari hasil seleksi fitur di atas, kemudian akan digunakan sebagai data model untuk memprediksi churn rate**

In [ ]:
model_df = final_data.select(['features','deposit_Index'])
model_df = model_df.withColumnRenamed("deposit_Index","label") # mengubah nama kolom supaya lebih mudah dimengerti
model_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



**Kemudian data model yang telah dibuat tadi akan dilakukan training dan testing, sehingga nantinya bisa menghasilkan nilai churn rate**

In [ ]:
training_df,test_df = model_df.randomSplit([0.75,0.25]) # 75% dari data untuk training dan 25% dari data untuk testing

**Yak lanjut aja dengan membuat model regresi logistik untuk melihat akurasi training dan testing kita**

In [ ]:
from pyspark.ml.classification import LogisticRegression # Prediksi menggunakan Metode Logistic Regression
log_reg = LogisticRegression().fit(training_df)

**Akhirnya sampai di penghujung kodingan wkwkwk. Daripada tambah pusing, langsung aja print akurasi hasil training dan testing**

In [ ]:
lr_summary = log_reg.summary
lr_summary.accuracy # Akurasi secara keseluruhan = 79%. Hasil akurasi biasanya berkorelasi dengan konsep underfitting/overfitting.
# Referensi bacaan mengenai konsep underfitting/overfitting: https://s.id/yhXPu 

0.7973215353342102

**Kita bandingkan dengan ambang batas area under ROC untuk mengetahui seberapa baik performa prediksi kita**

In [ ]:
# Referensi bacaan mengenai threshold ROC pada Logistic Regression: https://s.id/yheC6
lr_summary.areaUnderROC # Threshold ROC = 87%. Karena akurasi < Threshold, maka hasil prediksi belum baik.

0.8792386115297004

**And finally, last but not least, kita tampilkan hasil prediksi churn rate dari dataframe yang telah kita train dan test tadi**

In [ ]:
predictions = log_reg.transform(test_df)
predictions.select('label','prediction').show(10) # menampilkan 10 baris data teratas

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 10 rows



**Yap! Dapat dilihat bahwa hasil prediksi kita kurang sempurna. Namun yang penting, kita sudah berhasil tahu dan belajar mengenai langkah-langkah dari awal sampai akhir untuk menganalisis data menggunakan PySpark. Atas perhatiannya, saya ucapkan terima kasih :)**